In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/fruit-images-for-object-detection/train_zip/train'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/fruit-images-for-object-detection/train_zip/train/mixed_4.jpg
/kaggle/input/fruit-images-for-object-detection/train_zip/train/mixed_1.jpg
/kaggle/input/fruit-images-for-object-detection/train_zip/train/apple_29.xml
/kaggle/input/fruit-images-for-object-detection/train_zip/train/mixed_14.xml
/kaggle/input/fruit-images-for-object-detection/train_zip/train/apple_56.xml
/kaggle/input/fruit-images-for-object-detection/train_zip/train/banana_13.jpg
/kaggle/input/fruit-images-for-object-detection/train_zip/train/banana_27.jpg
/kaggle/input/fruit-images-for-object-detection/train_zip/train/mixed_5.xml
/kaggle/input/fruit-images-for-object-detection/train_zip/train/apple_25.jpg
/kaggle/input/fruit-images-for-object-detection/train_zip/train/banana_60.xml
/kaggle/input/fruit-images-for-object-detection/train_zip/train/banana_68.jpg
/kaggle/input/fruit-images-for-object-detection/train_zip/train/apple_49.xml
/kaggle/input/fruit-images-for-object-detection/train_zip/train/mixed_9.xml

In [2]:
# 필요한 모듈 임포트
import pandas as pd
import numpy as np
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import os
import cv2
import matplotlib.pyplot as plt
from keras import optimizers
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense,Activation,AveragePooling2D,MaxPooling2D,Dropout,BatchNormalization

Using TensorFlow backend.


In [3]:
# train을 train-val으로 나누기 위한 모듈 임포트
from sklearn.model_selection import train_test_split

np.random.seed(42)

In [4]:
# train set 불러와서 reshape 수행 후 train-val set 으로 나누기
# matplotlib은 RGB, 저장된 이미지는 BGR순이므로 cv2모듈을 이용해 바꿔주기
# 그래야 파란색 이미지로 안나옴
# 
from keras.preprocessing.image import ImageDataGenerator

train_images = []       
train_labels = []
shape = (200,200)  
train_path = '../input/fruit-images-for-object-detection/train_zip/train'

for filename in os.listdir('../input/fruit-images-for-object-detection/train_zip/train'):
    if filename.split('.')[1] == 'jpg':
        img = cv2.imread(os.path.join(train_path,filename))
        
        # Spliting file names and storing the labels for image in list
        train_labels.append(filename.split('_')[0])
        
        # Resize all images to a specific shape
        img = cv2.resize(img,shape)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        train_images.append(img)

# Converting labels into One Hot encoded sparse matrix
train_labels = pd.get_dummies(train_labels).values

# Converting train_images to array
x_train = np.array(train_images)


# # Splitting Training data into train and validation dataset
x_train,x_val,y_train,y_val = train_test_split(x_train,train_labels,random_state=42)

In [5]:
print(len(x_train),len(x_val))

180 60


In [6]:
print(x_train.shape)

(180, 200, 200, 3)


In [7]:
print(train_labels.shape)

(240, 4)


In [8]:
print(y_train.shape)

(180, 4)


In [9]:
# to increase the number of train images
datagen = ImageDataGenerator(rotation_range=20,
                                           width_shift_range=0.05,
                                           height_shift_range=0.05)
datagen.fit(x_train)
train_inc = datagen.flow(x_train, y_train,
                                    batch_size=16)

In [10]:
# test set 도 동일한 과정 거치기
test_images = []
test_labels = []
shape = (200,200)
test_path = '../input/fruit-images-for-object-detection/test_zip/test'

for filename in os.listdir('../input/fruit-images-for-object-detection/test_zip/test'):
    if filename.split('.')[1] == 'jpg':
        img = cv2.imread(os.path.join(test_path,filename))
        
        # Spliting file names and storing the labels for image in list
        test_labels.append(filename.split('_')[0])
        
        # Resize all images to a specific shape
        img = cv2.resize(img,shape)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        test_images.append(img)
        
# Converting test_images to array
x_test = np.array(test_images)

In [11]:
print(x_test.shape)

(60, 200, 200, 3)


In [12]:
# 모델 구축
model= Sequential()
model.add(Conv2D(kernel_size=(3,3), filters=32, activation='relu', input_shape=(200,200,3,)))

model.add(Conv2D(filters=32,kernel_size = (3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.4))

model.add(Conv2D(filters=64,kernel_size = (3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.4))

model.add(Conv2D(filters=64,kernel_size = (3,3),activation='relu'))

model.add(Flatten())

model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(4,activation = 'softmax'))
    
model.compile(
              loss='categorical_crossentropy', 
              metrics=['acc'],
              optimizer='adam'
             )

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 196, 196, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 98, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 96, 96, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 48, 48, 64)       

In [14]:
# 모델 훈련
# history = model.fit(x_train,train_labels,epochs=50,batch_size=50)
history=model.fit_generator(train_inc,epochs=70,steps_per_epoch=10,
                           verbose=1,validation_data=(x_val,y_val))

Epoch 1/70
10/10 [==============================] - 15s 2s/step - loss: 173.3216 - acc: 0.2365 - val_loss: 1.5155 - val_acc: 0.2833
Epoch 2/70
10/10 [==============================] - 14s 1s/step - loss: 1.5204 - acc: 0.2838 - val_loss: 1.3782 - val_acc: 0.3000
Epoch 3/70
10/10 [==============================] - 15s 1s/step - loss: 1.3941 - acc: 0.2500 - val_loss: 1.3827 - val_acc: 0.3500
Epoch 4/70
10/10 [==============================] - 14s 1s/step - loss: 1.3812 - acc: 0.3446 - val_loss: 1.3806 - val_acc: 0.3500
Epoch 5/70
10/10 [==============================] - 14s 1s/step - loss: 1.3838 - acc: 0.3041 - val_loss: 1.3779 - val_acc: 0.3500
Epoch 6/70
10/10 [==============================] - 13s 1s/step - loss: 1.3760 - acc: 0.3514 - val_loss: 1.3752 - val_acc: 0.3500
Epoch 7/70
10/10 [==============================] - 15s 1s/step - loss: 1.3704 - acc: 0.3649 - val_loss: 1.3723 - val_acc: 0.3500
Epoch 8/70
10/10 [==============================] - 14s 1s/step - loss: 1.3815 - acc: 0.

In [15]:
# Testing predictions and the actual label
checkImage = x_test
checklabel = test_labels

predict = model.predict(checkImage)
predict_arr=[]

output = { 0:'apple',1:'banana',2:'mixed',3:'orange'}
for i in range(len(x_test)):
    predict_arr.append(output[np.argmax(predict[i])])

In [16]:
#test 정확도
print(checklabel)
print("\n")
print(predict_arr)

['banana', 'banana', 'orange', 'apple', 'orange', 'orange', 'mixed', 'banana', 'orange', 'orange', 'banana', 'apple', 'apple', 'banana', 'banana', 'mixed', 'orange', 'apple', 'apple', 'apple', 'orange', 'banana', 'orange', 'banana', 'banana', 'apple', 'apple', 'orange', 'orange', 'apple', 'apple', 'orange', 'orange', 'orange', 'apple', 'banana', 'banana', 'orange', 'mixed', 'banana', 'banana', 'mixed', 'apple', 'banana', 'apple', 'banana', 'orange', 'apple', 'orange', 'banana', 'apple', 'mixed', 'apple', 'apple', 'banana', 'apple', 'orange', 'orange', 'banana', 'apple']


['banana', 'banana', 'banana', 'apple', 'orange', 'orange', 'mixed', 'banana', 'orange', 'orange', 'banana', 'apple', 'apple', 'banana', 'banana', 'orange', 'apple', 'apple', 'apple', 'apple', 'banana', 'banana', 'orange', 'banana', 'banana', 'apple', 'apple', 'banana', 'orange', 'apple', 'apple', 'banana', 'banana', 'orange', 'apple', 'banana', 'apple', 'banana', 'apple', 'banana', 'banana', 'banana', 'apple', 'banan

In [17]:
# test 정확도 수치화
print(np.mean(np.array(checklabel)==np.array(predict_arr)))

0.75
